In [1]:
import numpy as np
import sklearn.preprocessing as prep
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Xavier初始化器会根据某一层网络输入、输出节点数量自动调整最合适的分布
# Xavier初始化器让权重被初始化得不大不小，正好合适
# 数学角度上，Xavier就是让权重满足0均值，同时方差为 2/(n_in + n_out)，分布可以用均匀分布或者高斯分布
# 通过tf.random_uniform创建了一个(low, high)范围内的均匀分布
# fan_in 输入节点的数量 fan_out 输出节点的数量
def xavier_init(fan_in, fan_out, constant = 1):
    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval = low, maxval = high, dtype = tf.float32)

In [3]:
# 定义去噪自编码的class
class AdditiveFaussianNoiseAutoencoder(object):
    # 输入变量数、隐含层节点数、隐含层激活函数、优化器、高斯噪声系数
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus, 
                 optimizer=tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)
        self.training_scale = scale
        network_weights = self._initialize_weights()
        self.weights = network_weights
        
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        self.hidden = self.transfer(tf.add(tf.matmul(self.x + scale * tf.random_normal((n_input, )), self.weights['w1']), self.weights['b1']))
        self.reconstruction = tf.add(tf.matmul(self.hidden, self.weights['w2']), self.weights['b2'])
        
        self.cost = 0.5 * tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction, self.x), 2.0))
        self.optimizer = optimizer.minimize(self.cost)
        
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)
    # 参数初始化函数
    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xavier_init(self.n_input, self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden], dtype=tf.float32))
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden, self.n_input], dtype=tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input], dtype=tf.float32))
        return all_weights
    # 用一个batch数据进行训练兵返回当前的损失cost
    def partial_fit(self, X):
        cost, opt = self.sess.run((self.cost, self.optimizer), feed_dict={self.x: X, self.scale: self.training_scale})
        return cost
    
    def cal_total_cost(self, X):
        return self.sess.run(self.cost, feed_dict={self.x: X, self.scale: self.training_scale})
    
    def transform(self, X):
        return self.sess.run(self.hidden, feed_dict={self.x: X, self.scale: self.training_scale})
    
    def generate(self, hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights['b1'])
        return self.sess.run(self.reconstruction, feed_dict={self.hidden: hidden})
    
    def reconstruct(self, X):
        return self.sess.run(self.reconstruction, feed_dict={self.x: X, self.scale: self.training_scale})
    
    def getWeights(self):
        return self.sess.run(self.weights['w1'])
    
    def getBiases(self):
        return self.sess.run(self.weights['b1'])

In [4]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
# 对训练、测试数据进行标准化处理
# 标准化：即让数据变成0均值，且标准差为1的分布
# 方法就是 先减去均值，再除以标准差
def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X_train)  # 先在训练集上进行fit，得到一个共用的Scaler
    X_train = preprocessor.transform(X_train)
    X_test = preprocessor.transform(X_test)
    return X_train, X_test

In [6]:
# 定义一个获取随机block数据的函数：取一个从0到len(data)-batch_size之间的随机整数，再以这个随机数作为block的起始位置，然后顺序取到一个batch size的数据
def get_random_block_from_data(data, batch_size):
    start_index = np.random.randint(0, len(data) - batch_size)
    return data[start_index:(start_index + batch_size)]

In [7]:
X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)

In [8]:
# 总训练样本数
n_samples = int(mnist.train.num_examples)
# 最大训练的轮数
training_epochs = 20
# 每次的数量
batch_size = 128
# 设置每隔一轮就显示一次损失cost
display_step = 1

In [9]:
# 创建一个AGN自编码器的实例
autoencoder = AdditiveFaussianNoiseAutoencoder(n_input=784, n_hidden=200,
                                               transfer_function=tf.nn.softplus, 
                                               optimizer=tf.train.AdamOptimizer(learning_rate=0.001), 
                                               scale=0.01)
# 开始训练过程
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(n_samples / batch_size)
    for i in range(total_batch):
        batch_xs = get_random_block_from_data(X_train, batch_size)
        
        cost = autoencoder.partial_fit(batch_xs)
        avg_cost += cost / n_samples * batch_size
    
    if epoch % display_step == 0:
        print("Epoch: ", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

Epoch:  0001 cost= 17852.710422727
Epoch:  0002 cost= 11560.874592045
Epoch:  0003 cost= 10596.626080114
Epoch:  0004 cost= 10014.843424432
Epoch:  0005 cost= 9753.911777273
Epoch:  0006 cost= 9353.423552841
Epoch:  0007 cost= 9331.945791477
Epoch:  0008 cost= 9548.853823864
Epoch:  0009 cost= 8789.499659659
Epoch:  0010 cost= 8745.346554545
Epoch:  0011 cost= 9043.851969886
Epoch:  0012 cost= 8007.992773864
Epoch:  0013 cost= 7814.246480682
Epoch:  0014 cost= 7695.541155682
Epoch:  0015 cost= 9037.002271591
Epoch:  0016 cost= 8970.151557386
Epoch:  0017 cost= 7837.071840909
Epoch:  0018 cost= 7205.213496591
Epoch:  0019 cost= 8086.542142045
Epoch:  0020 cost= 8563.618600568


In [10]:
print("Total cost: " + str(autoencoder.cal_total_cost(X_test)))

Total cost: 696033.0
